In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv('yahoo_finance_articles.csv',usecols=['content'])
# df = pd.read_csv('/content/drive/MyDrive/yahoo_finance_articles.csv', usecols=['content'])

df.head()
df_0 = df.copy()

In [4]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
#tokenization
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string
import re

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s.]', '', text)
    text = ' '.join(text.split())
    return text

def tokenize_text(text):
    if pd.isna(text):
        return []
    clean_text = preprocess_text(text)
    tokens = word_tokenize(clean_text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens
             if token not in stop_words
             and token not in string.punctuation]
    return tokens

df['content_tokens'] = df['content'].apply(lambda x: tokenize_text(x))
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,content,content_tokens
0,In This Article: The Nasdaq led US stocks high...,"[article, nasdaq, led, us, stocks, higher, wed..."
1,In This Article: Netflix stock (NFLX) surged t...,"[article, netflix, stock, nflx, surged, alltim..."
2,In This Article: Nvidia (NVDA) stock jumped mo...,"[article, nvidia, nvda, stock, jumped, 4, wedn..."
3,In This Article: JPMorgan Chase (JPM) CEO Jami...,"[article, jpmorgan, chase, jpm, ceo, jamie, di..."
4,In This Article: By Stephen Nellis and Krystal...,"[article, stephen, nellis, krystal, hu, reuter..."


In [6]:
# # create a list of all unique tokens in the column 'content_tokens'
# unique_tokens = set()
# for tokens in df['content_tokens']:
#     unique_tokens.update(tokens)
# print(f'Number of unique tokens: {len(unique_tokens)}')
# print(f'Example of unique tokens: {list(unique_tokens)}')

In [8]:
# !pip install -U spacy
# !pip install -U spacy-transformers
# !python -m spacy download en_core_web_trf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [9]:
import spacy
nlp = spacy.load("en_core_web_trf")
text = "Tesla stock rises after strong earnings report. Elon Musk continues to lead the firm."
doc = nlp(text)
print("Extracted Named Entities:")
for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`.

Extracted Named Entities:
Tesla: ORG
Elon Musk: PERSON


In [10]:
def extract_named_entities(tokens):
    text = " ".join(tokens)
    doc = nlp(text)

    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

df['named_entities'] = df['content'].apply(extract_named_entities)
print(df[['content', 'named_entities']])

                                              content  \
0   In This Article: The Nasdaq led US stocks high...   
1   In This Article: Netflix stock (NFLX) surged t...   
2   In This Article: Nvidia (NVDA) stock jumped mo...   
3   In This Article: JPMorgan Chase (JPM) CEO Jami...   
4   In This Article: By Stephen Nellis and Krystal...   
5   In This Article: (Bloomberg) -- Investors will...   
6   In This Article: Johnson & Johnson (JNJ) beat ...   
7   (Bloomberg) -- JPMorgan Chase & Co.’s dealmake...   
8   In This Article: Hindenburg Research was widel...   
9   (Bloomberg) -- Venture Global Inc. slashed the...   
10  In This Article: (Reuters) - Goldman Sachs CEO...   
11  In This Article: LONDON (Reuters) - Shareholde...   
12  In This Article: A publicly traded coffee whol...   
13  In This Article: The artificial intelligence s...   
14  In This Article: DUBAI (Reuters) - Abu Dhabi-l...   
15  In This Article: (Bloomberg) -- Turkey’s sover...   
16  P&G Stopped Hiking Prices a

In [11]:
df['named_entities']

,named_entities
0,"[(1, CARDINAL), (3 %, PERCENT), (6 %, PERCENT)..."
1,"[(6 %, PERCENT), (7, CARDINAL), (5, CARDINAL),..."
2,"[($ 5 0, MONEY), (3, CARDINAL), (3, CARDINAL),..."
3,[]
4,[]
5,"[(1, MONEY), (9, CARDINAL), (5 8, CARDINAL), (..."
6,"[(4, CARDINAL), (5, CARDINAL), (5, CARDINAL), ..."
7,"[(4, CARDINAL), (2, CARDINAL)]"
8,[]
9,"[(3, CARDINAL), (4, CARDINAL), (3, CARDINAL), ..."


In [12]:
df['content_tokens_st'] = df['content_tokens'].apply(lambda x: ' '.join(x))
df['content_tokens_st']

,content_tokens_st
0,article nasdaq led us stocks higher wednesday ...
1,article netflix stock nflx surged alltime high...
2,article nvidia nvda stock jumped 4 wednesday e...
3,article jpmorgan chase jpm ceo jamie dimon wed...
4,article stephen nellis krystal hu reuters micr...
5,article bloomberg investors willing go market ...
6,article johnson johnson jnj beat fourth quarte...
7,bloomberg jpmorgan chase co.s dealmakers spend...
8,article hindenburg research widely recognized ...
9,bloomberg venture global inc. slashed marketed...


In [14]:
#Use a pre-trained NER model (e.g., SpaCy, BERT, or FinBERT) as a baseline.
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

df['entities'] = df['content'].apply(lambda x: extract_entities(x))

df['content_tokens_st_entities'] = df['content_tokens_st'].apply(lambda x: extract_entities(x))

display(df['entities'])
display(df['content_tokens_st_entities'])


,entities
0,"[(Nasdaq, ORG), (US, GPE), (Wednesday, DATE), ..."
1,"[(Netflix, ORG), (NFLX, ORG), (as much as 13.6..."
2,"[(Nvidia, ORG), (NVDA, ORG), (more than 4%, PE..."
3,"[(JPMorgan Chase, ORG), (JPM, ORG), (Jamie Dim..."
4,"[(Stephen Nellis, PERSON), (Krystal Hu, PERSON..."
5,"[(Bloomberg, ORG), (the coming months, DATE), ..."
6,"[(Johnson & Johnson, ORG), (JNJ, ORG), (fourth..."
7,"[(Bloomberg, ORG), (JPMorgan Chase & Co.’s, OR..."
8,"[(Hindenburg Research, ORG), (last week, DATE)..."
9,"[(Bloomberg, ORG), (Venture Global Inc., ORG),..."


,content_tokens_st_entities
0,"[(nasdaq, ORG), (us, GPE), (wednesday, DATE), ..."
1,"[(netflix, ORG), (13.6, CARDINAL), (wednesday,..."
2,"[(nvidia, ORG), (nvda, ORG), (4, CARDINAL), (w..."
3,"[(jamie dimon, PERSON), (wednesday, DATE), (tr..."
4,"[(stephen nellis, PERSON), (reuters, ORG), (mi..."
5,"[(bloomberg, PERSON), (coming months, DATE), (..."
6,"[(johnson johnson jnj, ORG), (fourth quarter, ..."
7,"[(jpmorgan, ORG), (swiss, NORP), (us, GPE), (b..."
8,"[(hindenburg, ORG), (last week, DATE), (one, C..."
9,"[(bloomberg venture global inc., ORG), (110 bi..."


In [15]:
df

,content,content_tokens,named_entities,content_tokens_st,entities,content_tokens_st_entities
0,In This Article: The Nasdaq led US stocks high...,"[article, nasdaq, led, us, stocks, higher, wed...","[(1, CARDINAL), (3 %, PERCENT), (6 %, PERCENT)...",article nasdaq led us stocks higher wednesday ...,"[(Nasdaq, ORG), (US, GPE), (Wednesday, DATE), ...","[(nasdaq, ORG), (us, GPE), (wednesday, DATE), ..."
1,In This Article: Netflix stock (NFLX) surged t...,"[article, netflix, stock, nflx, surged, alltim...","[(6 %, PERCENT), (7, CARDINAL), (5, CARDINAL),...",article netflix stock nflx surged alltime high...,"[(Netflix, ORG), (NFLX, ORG), (as much as 13.6...","[(netflix, ORG), (13.6, CARDINAL), (wednesday,..."
2,In This Article: Nvidia (NVDA) stock jumped mo...,"[article, nvidia, nvda, stock, jumped, 4, wedn...","[($ 5 0, MONEY), (3, CARDINAL), (3, CARDINAL),...",article nvidia nvda stock jumped 4 wednesday e...,"[(Nvidia, ORG), (NVDA, ORG), (more than 4%, PE...","[(nvidia, ORG), (nvda, ORG), (4, CARDINAL), (w..."
3,In This Article: JPMorgan Chase (JPM) CEO Jami...,"[article, jpmorgan, chase, jpm, ceo, jamie, di...",[],article jpmorgan chase jpm ceo jamie dimon wed...,"[(JPMorgan Chase, ORG), (JPM, ORG), (Jamie Dim...","[(jamie dimon, PERSON), (wednesday, DATE), (tr..."
4,In This Article: By Stephen Nellis and Krystal...,"[article, stephen, nellis, krystal, hu, reuter...",[],article stephen nellis krystal hu reuters micr...,"[(Stephen Nellis, PERSON), (Krystal Hu, PERSON...","[(stephen nellis, PERSON), (reuters, ORG), (mi..."
5,In This Article: (Bloomberg) -- Investors will...,"[article, bloomberg, investors, willing, go, m...","[(1, MONEY), (9, CARDINAL), (5 8, CARDINAL), (...",article bloomberg investors willing go market ...,"[(Bloomberg, ORG), (the coming months, DATE), ...","[(bloomberg, PERSON), (coming months, DATE), (..."
6,In This Article: Johnson & Johnson (JNJ) beat ...,"[article, johnson, johnson, jnj, beat, fourth,...","[(4, CARDINAL), (5, CARDINAL), (5, CARDINAL), ...",article johnson johnson jnj beat fourth quarte...,"[(Johnson & Johnson, ORG), (JNJ, ORG), (fourth...","[(johnson johnson jnj, ORG), (fourth quarter, ..."
7,(Bloomberg) -- JPMorgan Chase & Co.’s dealmake...,"[bloomberg, jpmorgan, chase, co.s, dealmakers,...","[(4, CARDINAL), (2, CARDINAL)]",bloomberg jpmorgan chase co.s dealmakers spend...,"[(Bloomberg, ORG), (JPMorgan Chase & Co.’s, OR...","[(jpmorgan, ORG), (swiss, NORP), (us, GPE), (b..."
8,In This Article: Hindenburg Research was widel...,"[article, hindenburg, research, widely, recogn...",[],article hindenburg research widely recognized ...,"[(Hindenburg Research, ORG), (last week, DATE)...","[(hindenburg, ORG), (last week, DATE), (one, C..."
9,(Bloomberg) -- Venture Global Inc. slashed the...,"[bloomberg, venture, global, inc., slashed, ma...","[(3, CARDINAL), (4, CARDINAL), (3, CARDINAL), ...",bloomberg venture global inc. slashed marketed...,"[(Bloomberg, ORG), (Venture Global Inc., ORG),...","[(bloomberg venture global inc., ORG), (110 bi..."


In [18]:
pd.set_option('display.max_colwidth', None)
df.content_tokens_st_entities

,content_tokens_st_entities
0,"[(nasdaq, ORG), (us, GPE), (wednesday, DATE), (netflix nflx, ORG), (us, GPE), (1.3, CARDINAL), (netflix sp, ORG), (0.6, CARDINAL), (0.3, CARDINAL), (500, CARDINAL), (tuesday, DATE), (us, GPE), (500 billion, CARDINAL), (bump techs, ORG), (oracle orcl, ORG), (roughly 7, CARDINAL), (tokyo softbank, ORG), (sftby, ORG), (11, CARDINAL), (netflix nflx, ORG), (11, CARDINAL), (streamer, ORG), (tuesday, DATE), (10, CARDINAL), (china, GPE), (feb. 1, DATE), (us, GPE), (tuesday, DATE), (china, GPE), (first, ORDINAL), (mexico canada, GPE), (china, GPE), (wednesday, DATE), (johnson johnson, PERSON), (quarterly, DATE), (procter, ORG), (pg, ORG), (amazon, ORG), (amzn, ORG), (nearly 2, CARDINAL), (wednesday, DATE), (234, CARDINAL), (seven, CARDINAL), (1.5, CARDINAL), (microsoft msft, ORG), (almost 3, CARDINAL), (nvidia nvda, ORG), (nearly 4, CARDINAL), (trump, PERSON), (donald trump, PERSON), (500 billion, MONEY), (us, GPE), (openai, ORG), (japanese, NORP), (softbank 9984.t, ORG), (oracle stock, ORG), (5, CARDINAL), (wednesday, DATE), (tuesdays, DATE), (cbs news, ORG), (first, ORDINAL), (nvidia, ORG), (4 wednesday, DATE), (ai, ORG), (c3.ai, ORG), (start wave, EVENT), (us, GPE), (coming weeks, DATE), (wedbush, ORG), (dan ives, PERSON), (tuesday, DATE), (night, TIME), (trump, ORG), (us, GPE), (china, GPE), (airelated, ORG), (wednesday, DATE), (netflix, ORG), (netflix, ORG), (nflx, ORG), (13, CARDINAL), (wednesday, DATE), (wall street, ORG), (streaming media companys, ORG), (fourth quarter, DATE), (1000, CARDINAL), (pivotal research, ORG), (1000, CARDINAL), (1250, CARDINAL), (18.9 million, CARDINAL), (fourth quarter, DATE), (quarterly, DATE), (companys, ORG), (netflix, ORG), (15 billion, CARDINAL), (fullyear, DATE), (tuesday, DATE), (netflix, ORG), (2025, DATE), (43.5 billion, CARDINAL), ...]"
1,"[(netflix, ORG), (13.6, CARDINAL), (wednesday, DATE), (wall street, ORG), (companys, ORG), (fourth quarter, DATE), (1000, CARDINAL), (pivotal research, ORG), (1000, CARDINAL), (18.9 million, CARDINAL), (fourth quarter, DATE), (quarterly, DATE), (companys, ORG), (q4, DATE), (jefferies, ORG), (james heaney, PERSON), (company, ORG), (15 billion, CARDINAL), (fullyear, DATE), (tuesday, DATE), (netflix, ORG), (2025, DATE), (43.5 billion, CARDINAL), (44.5 billion, CARDINAL), (43 billion, CARDINAL), (2024, DATE), (two, CARDINAL), (jake paul, PERSON), (mike tyson, PERSON), (squid game end, ORG), (print company, ORG), (7.99, CARDINAL), (6.99, CARDINAL), (15.49, CARDINAL), (2 24.99, CARDINAL), (8.99, CARDINAL), (1, CARDINAL), (wall street, ORG), (9.18 million, CARDINAL), (13.12 million, CARDINAL), (q4 2023, DATE), (last spring, DATE), (start year, DATE), (netflixs, ORG), (macquarie, ORG), (tim nollen, PERSON), (wednesday, DATE), (2024, DATE), (2025, DATE), (2026, DATE), (netflixs, ORG), (netflix, ORG), (greg peters, PERSON), (one, CARDINAL), (peters, PERSON), (quarter, DATE), (deutsche banks, ORG), (bryan kraft, ORG), (tyson, PERSON), (paul, PERSON), (nfl, ORG), (november, DATE), (paul, PERSON), (mike, PERSON), (tyson, PERSON), (108 million, CARDINAL), (global, NORP), (sporting event, EVENT), (2024, DATE), (super bowl, EVENT), (american, NORP), (124 million, CARDINAL), (us, GPE), (nfl games, EVENT), (around 30 million, CARDINAL), (netflix, ORG), (christmas day, EVENT), (us company, ORG), (ufc, ORG), (netflix, ORG), (cantmiss, ORG), (10.25 billion q4, MONEY), (bloomberg, ORG), (10.11 billion, CARDINAL), (16, CARDINAL), (last year, DATE), (netflix, ORG), (first quarter, DATE), (10.42 billion, CARDINAL), (10.48 billion, CARDINAL), (quarter, DATE), (4.27, CARDINAL), (4.18, CARDINAL), (2.11, CARDINAL), (yearago period, DATE), (netflix, ORG), (fourth quarter, DATE), (5.58, CARDINAL), (6.01, CARDINAL), (22.2, CARDINAL), ...]"
2,"[(nvidia, ORG), (nvda, ORG), (4, CARDINAL), (wednesday, DATE), (prior day, DATE), (trumps, PERSON), (500 billion, CARDINAL), (stargate, ORG), (softbank, ORG), (openai oracle, ORG), (orcl mgx openai, ORG), (nvidia well